### 1. Import thư viện

In [50]:
import requests
import json

### 2. Khai báo tên file

In [1]:
entries_file_name = r'D:\Learning\ML\BTL\get_raw_data\raw_data\entries.json'
puuids_of_100_player_having_most_matches_file_name = r'D:\Learning\ML\BTL\get_raw_data\raw_data\puuid.json'

### 3. Khai báo Header và API key cho Request

In [63]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0",
    "Accept-Language": "vi,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
}

In [99]:
api_key = 'RGAPI-535f85d1-32bf-4ba4-8c93-42a86252bd19'

### 4. Danh sách người chơi sở rank Kim Cương 1

In [ ]:
def get_Entries(api_key, headers):
    entries = []
    base_url = 'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/I'
    for i in range(26):
        url = base_url + '?page=' + str(i) + '&api_key=' + api_key
        response = requests.get(url, headers=headers)     
        data = response.json()      
        entries.extend(data)
    return entries

In [ ]:
entries = get_Entries(api_key, headers)

In [83]:
# Thêm tỷ lệ thắng cho mỗi entry
for player in entries:
    if isinstance(player, dict):
        total_games = player["wins"] + player["losses"]
        if total_games > 0:
            player["winRate"] = round((player["wins"] / total_games) * 100, 2)
        else:
            player["winRate"] = 10.0
    else:
        entries.remove(player)

# Sắp xếp entry theo số trận giảm dần
entries = sorted(entries, key=lambda x: x["wins"] + x["losses"], reverse=True)

In [ ]:
# Ghi dữ liệu
with open(entries_file_name, 'w', encoding='utf-8') as file:
    json.dump(entries, file, ensure_ascii=False, indent=4)

### 5. puuID của 100 người chơi nhiều trận nhất

In [ ]:
def get_puuid_of_100_player_having_most_matches(entries, api_key, headers):    
    summonerIds = []
    puuids = []
    base_url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/'
    for i in range(100):
        entry = entries[i]
        summonerIds.append(entry["summonerId"])
    for summonerId in summonerIds:
        url = base_url + summonerId + '?api_key=' + api_key
        response = requests.get(url, headers=headers)     
        data = response.json()  
        puuids.append(data["puuid"]) 
    return puuids

In [97]:
puuids_of_100_player_having_most_matches = get_puuid_of_100_player_having_most_matches(entries, api_key, headers)

In [ ]:
# Ghi dữ liệu
with open(puuids_of_100_player_having_most_matches_file_name, 'w', encoding='utf-8') as file:
    json.dump(puuids_of_100_player_having_most_matches, file, ensure_ascii=False, indent=4)